# پروژه سیستم توصیه‌گر فیلم (IMDB/MovieLens)
در این پروژه یک سیستم توصیه‌گر با **دو روش تدریس‌شده** پیاده‌سازی می‌کنیم:

1) **Content-Based Filtering**  
   - استفاده از **ژانر فیلم‌ها** (one-hot)  
   - ساخت **User Profile** و پیشنهاد فیلم‌های جدید

2) **Collaborative Filtering (Memory-Based)**  
   - **User-Based** با **Pearson Correlation**
   - **Item-Based** (برای مقیاس‌پذیری) با KNN روی ماتریس آیتم–کاربر

در پایان، یک روش **Hybrid** نیز ارائه می‌شود.

فایل‌ها:
- `movies.csv`: movieId, title, genres
- `ratings.csv`: userId, movieId, rating, timestamp


## 0) تنظیمات و کتابخانه‌ها
- pandas/numpy برای پردازش داده
- sklearn برای KNN و محاسبات کمکی


In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_colwidth", 100)

## 1) بارگذاری داده‌ها
اگر Kernel ریستارت شود، حتماً این سلول باید دوباره اجرا شود.


In [2]:
MOVIES_PATH = "movies.csv"
RATINGS_PATH = "ratings.csv"

movies = pd.read_csv(MOVIES_PATH)
ratings = pd.read_csv(RATINGS_PATH)

print("movies shape :", movies.shape)
print("ratings shape:", ratings.shape)

display(movies.head())
display(ratings.head())

movies shape : (9742, 3)
ratings shape: (100836, 4)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


## 2) کنترل کیفیت داده‌ها (QC)
- بررسی مقدارهای گمشده
- بررسی ردیف‌های تکراری


In [3]:
print("Missing values (movies):")
display(movies.isna().sum())

print("\nMissing values (ratings):")
display(ratings.isna().sum())

print("\nDuplicates:")
print("movies duplicates :", movies.duplicated().sum())
print("ratings duplicates:", ratings.duplicated().sum())

Missing values (movies):


movieId    0
title      0
genres     0
dtype: int64


Missing values (ratings):


userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


Duplicates:
movies duplicates : 0
ratings duplicates: 0


## 3) EDA کوتاه
- تعداد کاربران یکتا
- تعداد فیلم‌های یکتا
- توصیف آماری rating

In [5]:
print("Unique users :", ratings["userId"].nunique())
print("Unique movies:", ratings["movieId"].nunique())
print("Total ratings:", len(ratings))

display(ratings["rating"].describe())
display(ratings["rating"].value_counts().sort_index())

Unique users : 610
Unique movies: 9724
Total ratings: 100836


count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64

# بخش A) Content-Based Filtering

مطابق تدریس:
- ژانرهای هر فیلم را به ویژگی‌های باینری (one-hot) تبدیل می‌کنیم.
- برای هر کاربر یک **User Profile** می‌سازیم (میانگین وزن‌دار ژانرها با rating).
- سپس فیلم‌های ندیده را بر اساس شباهت/امتیاز به پروفایل کاربر رتبه‌بندی می‌کنیم.

## A1) تبدیل ژانرها به one-hot
در `genres` ژانرها با `|` جدا شده‌اند.


In [6]:
movies_cb = movies.copy()

# تبدیل ژانرها به لیست
movies_cb["genres_list"] = movies_cb["genres"].fillna("").str.split("|")

# استخراج کل ژانرها
all_genres = sorted({g for gs in movies_cb["genres_list"] for g in gs if g and g != "(no genres listed)"})
print("Number of genres:", len(all_genres))
print(all_genres[:15], "...")

Number of genres: 19
['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance'] ...


In [7]:
# ساخت one-hot genres
for g in all_genres:
    movies_cb[g] = movies_cb["genres_list"].apply(lambda x: int(g in x))

genre_cols = all_genres  # ستون‌های ویژگی
movies_cb[["movieId", "title", "genres"] + genre_cols[:8]].head()

,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,1,0,0,1
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0


## A2) ساخت پروفایل کاربر از روی ratingهای خودش
تابع زیر برای یک `userId`:
1) فیلم‌های امتیاز داده شده را می‌گیرد
2) ماتریس ژانرهای آن فیلم‌ها را برمی‌دارد
3) میانگین وزن‌دار با rating می‌سازد (User Profile)
4) به فیلم‌های ندیده امتیاز می‌دهد و Top-N پیشنهاد می‌کند

In [8]:
# نگاشت movieId -> index در movies_cb
mid_to_row = pd.Series(movies_cb.index, index=movies_cb["movieId"]).to_dict()

def recommend_content_based_for_user(user_id, top_n=10, min_ratings=5):
    user_df = ratings[ratings["userId"] == user_id][["movieId", "rating"]].copy()
    if user_df.empty:
        raise ValueError("این userId در ratings وجود ندارد یا هیچ امتیازی ثبت نکرده است.")
    if len(user_df) < min_ratings:
        raise ValueError(f"این کاربر فقط {len(user_df)} امتیاز دارد؛ برای پروفایل بهتر حداقل {min_ratings} لازم است.")

    # فقط movieId های موجود در movies
    user_df = user_df[user_df["movieId"].isin(mid_to_row.keys())].copy()
    if user_df.empty:
        raise ValueError("هیچ movieId معتبر از این کاربر در movies.csv یافت نشد.")

    seen_movie_ids = set(user_df["movieId"].tolist())

    # ماتریس ژانرهای فیلم‌های دیده شده
    idx = [mid_to_row[m] for m in user_df["movieId"]]
    user_genre_matrix = movies_cb.loc[idx, genre_cols].values  # (k, G)

    # وزن‌ها (rating)
    w = user_df["rating"].values.astype(float)
    w = w / (w.sum() + 1e-9)

    # User Profile: (G,)
    user_profile = (user_genre_matrix.T @ w)  # (G,)

    # امتیازدهی به همه فیلم‌ها (dot product)
    all_movie_scores = movies_cb[genre_cols].values @ user_profile  # (num_movies,)

    # حذف دیده‌شده‌ها
    candidates = movies_cb[~movies_cb["movieId"].isin(seen_movie_ids)][["movieId", "title", "genres"]].copy()
    candidates["score"] = all_movie_scores[candidates.index]

    # خروجی Top-N
    return candidates.sort_values("score", ascending=False).head(top_n).reset_index(drop=True)

# تست
recommend_content_based_for_user(user_id=1, top_n=10)

,movieId,title,genres,score
0,81132,Rubber (2010),Action|Adventure|Comedy|Crime|Drama|Film-Noir|Horror|Mystery|Thriller|Western,1.992103
1,117646,Dragonheart 2: A New Beginning (2000),Action|Adventure|Comedy|Drama|Fantasy|Thriller,1.831194
2,71999,Aelita: The Queen of Mars (Aelita) (1924),Action|Adventure|Drama|Fantasy|Romance|Sci-Fi|Thriller,1.758144
3,4956,"Stunt Man, The (1980)",Action|Adventure|Comedy|Drama|Romance|Thriller,1.742349
4,4719,Osmosis Jones (2001),Action|Animation|Comedy|Crime|Drama|Romance|Thriller,1.701876
5,164226,Maximum Ride (2016),Action|Adventure|Comedy|Fantasy|Sci-Fi|Thriller,1.693978
6,6902,Interstate 60 (2002),Adventure|Comedy|Drama|Fantasy|Mystery|Sci-Fi|Thriller,1.688055
7,52462,Aqua Teen Hunger Force Colon Movie Film for Theaters (2007),Action|Adventure|Animation|Comedy|Fantasy|Mystery|Sci-Fi,1.677196
8,546,Super Mario Bros. (1993),Action|Adventure|Children|Comedy|Fantasy|Sci-Fi,1.657453
9,55116,"Hunting Party, The (2007)",Action|Adventure|Comedy|Drama|Thriller,1.631787


## A3) توصیه «فیلم مشابه فیلم» در Content-Based (بر اساس ژانر)
در اینجا شباهت دو فیلم را با **کسینوسی روی بردار one-hot ژانر** می‌سنجیم.

In [9]:
def recommend_similar_by_genres(movie_title, top_n=10):
    # پیدا کردن فیلم
    matches = movies_cb[movies_cb["title"] == movie_title]
    if matches.empty:
        raise ValueError("عنوان دقیق فیلم در movies.csv پیدا نشد.")
    row_idx = matches.index[0]

    X = movies_cb[genre_cols].values.astype(float)
    v = X[row_idx].reshape(1, -1)

    # cosine similarity دستی (چون one-hot است)
    v_norm = np.linalg.norm(v) + 1e-9
    X_norm = np.linalg.norm(X, axis=1) + 1e-9
    sims = (X @ v.ravel()) / (X_norm * v_norm)

    # انتخاب مشابه‌ها
    order = np.argsort(-sims)
    order = [i for i in order if i != row_idx][:top_n]

    out = movies_cb.loc[order, ["movieId", "title", "genres"]].copy()
    out["similarity"] = sims[order]
    return out.reset_index(drop=True)

recommend_similar_by_genres("Toy Story (1995)", top_n=10)

,movieId,title,genres,similarity
0,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,1.0
1,103755,Turbo (2013),Adventure|Animation|Children|Comedy|Fantasy,1.0
2,2294,Antz (1998),Adventure|Animation|Children|Comedy|Fantasy,1.0
3,91355,Asterix and the Vikings (Astérix et les Vikings) (2006),Adventure|Animation|Children|Comedy|Fantasy,1.0
4,65577,"Tale of Despereaux, The (2008)",Adventure|Animation|Children|Comedy|Fantasy,1.0
5,136016,The Good Dinosaur (2015),Adventure|Animation|Children|Comedy|Fantasy,1.0
6,53121,Shrek the Third (2007),Adventure|Animation|Children|Comedy|Fantasy,1.0
7,3754,"Adventures of Rocky and Bullwinkle, The (2000)",Adventure|Animation|Children|Comedy|Fantasy,1.0
8,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,1.0
9,4016,"Emperor's New Groove, The (2000)",Adventure|Animation|Children|Comedy|Fantasy,1.0


# بخش B) Collaborative Filtering (Memory-Based)

مطابق جزوه:
- **Memory-Based** شامل:
  1) **User-Based** (Pearson Correlation)
  2) **Item-Based** (برای مقیاس‌پذیری)

در این نوت‌بوک هر دو را پیاده‌سازی می‌کنیم.

## B1) ساخت ماتریس User–Item
- سطر: userId
- ستون: movieId
- مقدار: rating

In [10]:
user_item = ratings.pivot_table(index="userId", columns="movieId", values="rating")
print("User-Item shape:", user_item.shape)

user_item.iloc[:5, :10]

User-Item shape: (610, 9724)


movieId,1,2,3,4,5,6,7,8,9,10
userId,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## B2) User-Based Collaborative Filtering با Pearson

ایده:
- شباهت کاربران بر اساس Pearson و روی آیتم‌های مشترک
- پیش‌بینی امتیاز فیلم‌های ندیده با میانگین وزن‌دار rating کاربران مشابه

برای جلوگیری از خطا/کندی:
- حداقل تعداد فیلم مشترک (`min_common`) تعیین می‌کنیم
- فقط Top-K کاربر مشابه استفاده می‌کنیم

> **نکته مهم:** این پیاده‌سازی از نوع **Memory-Based Collaborative Filtering** است.  
> به دلیل محاسبه شباهت کاربر هدف با سایر کاربران (حلقه روی کاربران)،  
> این روش از نظر **Scalability** برای دیتاست‌های بسیار بزرگ محدودیت دارد،  
> که این موضوع یکی از چالش‌های اصلی سیستم‌های توصیه‌گر مطابق جزوه درس است.


In [11]:
def pearson_similarity(u, v):
    """
    u و v: سری‌های pandas از rating برای دو کاربر (روی فیلم‌های مشترک)
    """
    if len(u) < 2:
        return np.nan
    u_mean = u.mean()
    v_mean = v.mean()
    num = ((u - u_mean) * (v - v_mean)).sum()
    den = np.sqrt(((u - u_mean) ** 2).sum()) * np.sqrt(((v - v_mean) ** 2).sum())
    if den == 0:
        return np.nan
    return num / den


def recommend_user_based_pearson(user_id, top_n=10, k_sim_users=30, min_common=10, min_sim=0.1):
    if user_id not in user_item.index:
        raise ValueError("این userId در داده وجود ندارد.")

    target = user_item.loc[user_id]
    target_rated = target.dropna()
    if target_rated.empty:
        raise ValueError("این کاربر هیچ rating ندارد (Cold Start).")

    # پیدا کردن کاربران با حداقل فیلم مشترک
    candidates = []
    for other in user_item.index:
        if other == user_id:
            continue
        other_rated = user_item.loc[other].dropna()

        common = target_rated.index.intersection(other_rated.index)
        if len(common) < min_common:
            continue

        sim = pearson_similarity(target_rated.loc[common], other_rated.loc[common])
        if pd.isna(sim) or sim < min_sim:
            continue
        candidates.append((other, sim))

    if not candidates:
        raise ValueError("کاربر مشابه کافی پیدا نشد (مشترکات کم/شباهت پایین).")

    # Top-K مشابه
    candidates = sorted(candidates, key=lambda x: x[1], reverse=True)[:k_sim_users]
    sim_users = dict(candidates)

    seen = set(target_rated.index.tolist())

    # پیش‌بینی برای فیلم‌های ندیده: مجموع وزن‌دار
    weighted_sum = {}
    sim_sum = {}

    for other, sim in sim_users.items():
        other_r = user_item.loc[other].dropna()
        for mid, r in other_r.items():
            if mid in seen:
                continue
            weighted_sum[mid] = weighted_sum.get(mid, 0.0) + sim * r
            sim_sum[mid] = sim_sum.get(mid, 0.0) + sim

    if not weighted_sum:
        raise ValueError("پس از تجمیع، هیچ پیشنهاد معناداری ساخته نشد.")

    preds = [(mid, weighted_sum[mid] / (sim_sum[mid] + 1e-9)) for mid in weighted_sum]
    preds = sorted(preds, key=lambda x: x[1], reverse=True)[:top_n]

    out = movies[movies["movieId"].isin([m for m, _ in preds])][["movieId", "title", "genres"]].copy()
    out["pred_rating"] = out["movieId"].map(dict(preds))
    out = out.sort_values("pred_rating", ascending=False).reset_index(drop=True)
    return out

recommend_user_based_pearson(user_id=1, top_n=10, k_sim_users=30, min_common=10, min_sim=0.1)

,movieId,title,genres,pred_rating
0,1230,Annie Hall (1977),Comedy|Romance,5.0
1,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,5.0
2,168252,Logan (2017),Action|Sci-Fi,5.0
3,915,Sabrina (1954),Comedy|Romance,5.0
4,2583,Cookie's Fortune (1999),Comedy|Drama,5.0
5,112175,How to Train Your Dragon 2 (2014),Action|Adventure|Animation,5.0
6,146662,Dragons: Gift of the Night Fury (2011),Adventure|Animation|Comedy,5.0
7,319,Shallow Grave (1994),Comedy|Drama|Thriller,5.0
8,112,Rumble in the Bronx (Hont faan kui) (1995),Action|Adventure|Comedy|Crime,5.0
9,737,Barb Wire (1996),Action|Sci-Fi,5.0


## B3) Item-Based Collaborative Filtering با KNN (Cosine Distance)

برای مقیاس‌پذیری (Scalability)، به جای ساخت ماتریس شباهت کامل آیتم‌ها،
از KNN استفاده می‌کنیم تا برای هر فیلم فقط نزدیک‌ترین همسایه‌ها را پیدا کنیم.

In [12]:
# ماتریس آیتم–کاربر با پر کردن NaN با صفر (برای cosine)
user_item_filled = user_item.fillna(0)
item_user = user_item_filled.T  # rows=movieId, cols=userId

knn_item = NearestNeighbors(metric="cosine", algorithm="brute")
knn_item.fit(item_user.values)

movie_ids = item_user.index.to_numpy()
movieid_to_pos = {mid: i for i, mid in enumerate(movie_ids)}
pos_to_movieid = {i: mid for i, mid in enumerate(movie_ids)}

def recommend_item_based_knn(user_id, top_n=10, k_neighbors=30, max_seed_movies=15):
    """
    توصیه به کاربر با Item-Based:
    - چند فیلمِ با rating بالاتر از کاربر را به عنوان Seed می‌گیریم (برای سرعت)
    - همسایه‌های هر Seed را با KNN می‌گیریم
    - امتیاز نهایی: مجموع وزن‌دار similarity * rating
    """
    if user_id not in user_item.index:
        raise ValueError("این userId وجود ندارد.")

    seen = user_item.loc[user_id].dropna()
    if seen.empty:
        raise ValueError("این کاربر هیچ rating ندارد (Cold Start).")

    # Seed ها: بهترین امتیازها برای سرعت/کیفیت
    seeds = seen.sort_values(ascending=False).head(max_seed_movies)

    scores = {}
    sim_sums = {}
    seen_mids = set(seen.index.tolist())

    for mid, r in seeds.items():
        if mid not in movieid_to_pos:
            continue

        pos = movieid_to_pos[mid]
        distances, indices = knn_item.kneighbors(item_user.values[pos].reshape(1, -1), n_neighbors=k_neighbors)

        for dist, neigh_pos in zip(distances.ravel(), indices.ravel()):
            neigh_mid = pos_to_movieid[neigh_pos]
            if neigh_mid in seen_mids:
                continue
            sim = 1 - dist
            if sim <= 0:
                continue
            scores[neigh_mid] = scores.get(neigh_mid, 0.0) + sim * r
            sim_sums[neigh_mid] = sim_sums.get(neigh_mid, 0.0) + sim

    if not scores:
        raise ValueError("هیچ پیشنهاد معناداری ساخته نشد (احتمالاً داده کم است).")

    preds = [(m, scores[m] / (sim_sums[m] + 1e-9)) for m in scores]
    preds = sorted(preds, key=lambda x: x[1], reverse=True)[:top_n]

    out = movies[movies["movieId"].isin([m for m, _ in preds])][["movieId", "title", "genres"]].copy()
    out["pred_rating"] = out["movieId"].map(dict(preds))
    out = out.sort_values("pred_rating", ascending=False).reset_index(drop=True)
    return out

recommend_item_based_knn(user_id=1, top_n=10, k_neighbors=20, max_seed_movies=10)

,movieId,title,genres,pred_rating
0,380,True Lies (1994),Action|Adventure|Comedy|Romance|Thriller,5.0
1,318,"Shawshank Redemption, The (1994)",Crime|Drama,5.0
2,778,Trainspotting (1996),Comedy|Crime|Drama,5.0
3,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,5.0
4,104,Happy Gilmore (1996),Comedy,5.0
5,153,Batman Forever (1995),Action|Adventure|Comedy|Crime,5.0
6,293,Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Action|Crime|Drama|Thriller,5.0
7,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,5.0
8,19,Ace Ventura: When Nature Calls (1995),Comedy,5.0
9,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,5.0


# بخش C) چالش‌ها (طبق جزوه)

1) **Data Sparsity**: ماتریس user–item خلوت است و بیشتر خانه‌ها خالی‌اند.  
2) **Cold Start**: کاربر/آیتم جدید اطلاعات کافی برای توصیه ندارد.  
3) **Scalability**: با افزایش کاربران و فیلم‌ها محاسبات سنگین می‌شود.

راهکار متداول: **Hybrid Solution** (ترکیب Content-Based و Collaborative).

## C1) Hybrid Recommendation
امتیاز نهایی ترکیبی:
`final = alpha * CF_score + (1-alpha) * CB_score`

در اینجا:
- `CB_score` از Content-Based (User Profile) می‌آید
- `CF_score` از Item-Based KNN می‌آید (پایدارتر و سریع‌تر)

In [13]:
def hybrid_recommendation(user_id, top_n=10, alpha=0.7):
    """
    alpha نزدیک 1 => تاکید بیشتر روی Collaborative
    alpha نزدیک 0 => تاکید بیشتر روی Content-Based
    """
    # Content-Based candidates (بیشتر می‌گیریم تا ترکیب بهتر شود)
    cb = recommend_content_based_for_user(user_id=user_id, top_n=300)
    cb_scores = cb.set_index("movieId")["score"].to_dict()

    # Item-based CF candidates
    cf = recommend_item_based_knn(user_id=user_id, top_n=300, k_neighbors=20, max_seed_movies=10)
    cf_scores = cf.set_index("movieId")["pred_rating"].to_dict()

    all_mids = set(cb_scores) | set(cf_scores)

    # نرمال‌سازی min-max برای هم‌مقیاس شدن
    def minmax(d):
        if not d:
            return {}
        v = np.array(list(d.values()))
        mn, mx = v.min(), v.max()
        if mx - mn < 1e-9:
            return {k: 0.0 for k in d}
        return {k: (val - mn) / (mx - mn) for k, val in d.items()}

    cb_n = minmax(cb_scores)
    cf_n = minmax(cf_scores)

    final = []
    for mid in all_mids:
        s = alpha * cf_n.get(mid, 0.0) + (1 - alpha) * cb_n.get(mid, 0.0)
        final.append((mid, s))

    final = sorted(final, key=lambda x: x[1], reverse=True)[:top_n]

    out = movies[movies["movieId"].isin([m for m, _ in final])][["movieId", "title", "genres"]].copy()
    out["final_score"] = out["movieId"].map(dict(final))
    out = out.sort_values("final_score", ascending=False).reset_index(drop=True)
    return out

hybrid_recommendation(user_id=1, top_n=10, alpha=0.7)

,movieId,title,genres,final_score
0,380,True Lies (1994),Action|Adventure|Comedy|Romance|Thriller,0.800592
1,318,"Shawshank Redemption, The (1994)",Crime|Drama,0.660902
2,153,Batman Forever (1995),Action|Adventure|Comedy|Crime,0.653309
3,778,Trainspotting (1996),Comedy|Crime|Drama,0.629103
4,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,0.624874
5,104,Happy Gilmore (1996),Comedy,0.622236
6,293,Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Action|Crime|Drama|Thriller,0.600672
7,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,0.595364
8,19,Ace Ventura: When Nature Calls (1995),Comedy,0.593128
9,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,0.590459


## نتیجه‌گیری
در این نوت‌بوک (مطابق جزوه و لَب‌ها):
- Content-Based با one-hot ژانر و User Profile پیاده شد.
- Collaborative Filtering (Memory-Based) شامل:
  - User-Based Pearson
  - Item-Based KNN (Cosine)
- چالش‌ها (Sparsity, Cold Start, Scalability) توضیح داده شد.
- Hybrid برای بهبود پیشنهادها ارائه شد.

برای نمره کامل:
- اجرای بدون خطا (Restart & Run All)
- توضیح مفهومی در Markdown
- نمایش خروجی چند توصیه در هر بخش


### پایان